# Creating a grid based on GeoHashes
https://blog.tafkas.net/2018/09/28/creating-a-grid-based-on-geohashes/

In [11]:
import geohash

precision=7

def is_geohash_in_bounding_box(current_geohash, bbox_coordinates):
    """Checks if the box of a geohash is inside the bounding box

    :param current_geohash: a geohash
    :param bbox_coordinates: bounding box coordinates
    :return: true if the the center of the geohash is in the bounding box
    """

    coordinates = geohash.decode(current_geohash)
    geohash_in_bounding_box = (bbox_coordinates[0] < coordinates[0] < bbox_coordinates[2]) and (
            bbox_coordinates[1] < coordinates[1] < bbox_coordinates[3])
    return geohash_in_bounding_box

def build_geohash_box(current_geohash):
    """Returns a GeoJSON Polygon for a given geohash

    :param current_geohash: a geohash
    :return: a list representation of th polygon
    """

    b = geohash.bbox(current_geohash)
    
    polygon = {}
    polygon['type'] = 'Feature'
    polygon = []
    polygon['coordinates'] = [(b['w'], b['s']), (b['w'], b['n']), (b['e'], b['n']), (b['e'], b['s'],), (b['w'], b['s'])]
    polygon['type'] = "Polygon"
    return polygon

def compute_geohash_tiles(bbox_coordinates, precision):
    """Computes all geohash tile in the given bounding box

    :param bbox_coordinates: the bounding box coordinates of the geohashes
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding box, assuming the earth is flat ;)
    center_latitude = (bbox_coordinates[0] + bbox_coordinates[2]) / 2
    center_longitude = (bbox_coordinates[1] + bbox_coordinates[3]) / 2

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=precision)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            if neighbor not in checked_geohashes and is_geohash_in_bounding_box(neighbor, bbox_coordinates):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes

In [38]:
import geojson
from geojson import MultiLineString
from geojson import MultiPolygon
geojson.Fe

def write_geohash_layer(geohashes):
    """Writes a grid layer based on the geohashes

    :param geohashes: a list of geohashes
    """
    
    layer = MultiPolygon([build_geohash_box(gh) for gh in geohashes])
    with open('ghash_berlin_bbox.json', 'wb') as f:
        f.write(geojson.dumps(layer, sort_keys=True).encode('utf-8'))

In [39]:
x = compute_geohash_tiles([52.338261, 13.08835,52.67551, 13.76116], precision=5)

In [40]:
len(x)

120

In [43]:
b = geohash.bbox('9zuzz')
b

{'s': 44.9560546875, 'w': -94.2626953125, 'n': 45.0, 'e': -94.21875}

In [56]:
polygon = {}
polygon['type'] = 'Feature'
polygon['properties'] = {"name": "current_geohash", "Device Count": "devicecount"}
polygon

{'type': 'Feature',
 'properties': {'name': 'current_geohash', 'Device Count': 'devicecount'}}

In [46]:
geojson.FeatureCollection()

geojson.feature.FeatureCollection

In [41]:
write_geohash_layer(x)

In [32]:
import pandas as pd

counts = pd.read_table('setagaya_geohash8_6mo_counts.tsv', header=None)

In [54]:
counts_sample = counts.head()
counts.head()

,0,1
0,2jtb69xt,2
1,2jtb69xv,1
2,2jtb69xy,1
3,2jtb6cb1,1
4,2jtb6cbc,1


In [55]:
counts_sample['new_col'] = list(zip(counts_sample[0],counts_sample[1]))
counts_sample

/Users/bcoates/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,0,1,new_col
0,2jtb69xt,2,"(2jtb69xt, 2)"
1,2jtb69xv,1,"(2jtb69xv, 1)"
2,2jtb69xy,1,"(2jtb69xy, 1)"
3,2jtb6cb1,1,"(2jtb6cb1, 1)"
4,2jtb6cbc,1,"(2jtb6cbc, 1)"


In [67]:
newlist = [[gh,ghc] for gh,ghc in counts_sample['new_col']]
newlist

[['2jtb69xt', 2],
 ['2jtb69xv', 1],
 ['2jtb69xy', 1],
 ['2jtb6cb1', 1],
 ['2jtb6cbc', 1]]

In [72]:
gh = newlist[0][0]
ghc = newlist[0][1]
gh

'2jtb69xt'

In [73]:
ghc

2

In [35]:
counts2 = counts.sort_values(by=[1], ascending=False)

In [36]:
write_geohash_layer(counts[0].head(10000))

# Convert geohash to match shape of geojson

In [6]:
from shapely import geometry

def compute_geohash_tiles_from_polygon(polygon, precision):
    """Computes all hex tile in the given polygon

    :param polygon: the polygon
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding, assuming the earth is flat ;)
    center_latitude = polygon.centroid.coords[0][1]
    center_longitude = polygon.centroid.coords[0][0]

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=precision)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            point = geometry.Point(geohash.decode(neighbor)[::-1])
            if neighbor not in checked_geohashes and polygon.contains(point):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes